In [6]:
import json
import imageio
import numpy as np
import os
import re
from skimage import draw, io
import uuid

In [7]:
# labelbox_json = json.load(open('./rocks.json'))
labelbox_json = json.load(open('geo_solution_rocks.json'))
output_path = './output'
resize = True

In [8]:
def resize_image_to_square(image):
    image_dtype = image.dtype
    
    h, w = image.shape[:2]
    window = (0, 0, h, w)
    padding = [(0, 0), (0, 0), (0, 0)]

    max_dim = max(h, w)

    top_pad = (max_dim - h) // 2
    bottom_pad = max_dim - h - top_pad
    left_pad = (max_dim - w) // 2
    right_pad = max_dim - w - left_pad
    padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
    image = np.pad(image, padding, mode='constant', constant_values=0)
    
    print(padding)
    return image.astype(image_dtype), padding

def resize_mask(mask, padding):
    return np.pad(mask, padding, mode='constant', constant_values=0)


In [9]:
for rock in labelbox_json:
    image_name = rock['External ID']
    image_url = rock['Labeled Data']
    print("image_name: {}".format(image_name))
    print("image_url: {}".format(image_url))
    
    image = io.imread(image_url)
    if resize:
        image, padding = resize_image_to_square(image)
    
    height, width = image.shape[:2]
    
    labels = rock['Label']
    if labels == 'Skip':
        continue
        
    for label_name, label_polygons in labels.items():
        print(label_name)
#         mask = np.zeros([height, width, len(label_polygons)], dtype=np.uint8)
        
        for label_polygon in label_polygons:
            mask = np.zeros((height, width), dtype=np.uint8)
#             print(mask.shape)
            all_points_x = []
            all_points_y = []
            for coords in label_polygon:
                all_points_x.append(coords['x'])
                all_points_y.append(coords['y'])

#             print(all_points_x)
#             print(all_points_y)
            rr, cc = draw.polygon(all_points_y, all_points_x)
            
            mask[rr, cc] = 1
            mask = np.flipud(mask)
            if resize:
                mask = resize_mask(mask, padding[:2])
                
#             print(mask)

            uuid_hex = uuid.uuid4().hex
            image_output_path = os.path.join(output_path, 'images', image_name + uuid_hex + '.png')
            imageio.imwrite(image_output_path, image)
            
            instance_mask_output_path = os.path.join(output_path, 'masks', image_name + uuid_hex + '.png')
            imageio.imwrite(instance_mask_output_path, mask)
            
    if 'Masks' in rock.keys():
        binary_masks = []
        for mask_name in rock['Masks'].keys():
            binary_masks.append({
                'Name': mask_name,
                'Url': rock['Masks'][mask_name]
            })
        print("{}, {}".format(image_name, binary_masks))
    print("")

image_name: rock-1.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F9530069e-4db0-4b21-83b0-00204c20d0f2%2Frock-1.png?alt=media&token=47a7e49c-6633-4987-b57e-8eeba01015af
[(552, 552), (0, 0), (0, 0)]
image_name: CE072_0007_01.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F5da96c03-8bd1-4f8e-865b-ef452a085bd2%2FCE072_0007_01.png?alt=media&token=3e1b269a-06b5-44ff-b01e-fedadaf755ba
[(9310, 9310), (0, 0), (0, 0)]
image_name: CE072_0007_02.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F5da96c03-8bd1-4f8e-865b-ef452a085bd2%2FCE072_0007_02.png?alt=media&token=bd52f33d-4ebe-4fd8-a6f7-31ae40c029d4
[(9310, 9310), (0, 0), (0, 0)]
Vein Type 1

image_name: CE072_0009_01.png
image_url: https://firebasestorage.googleapis.com/v0/b/labelbox-193903.appspot.com/o/cjh1icgkd8lak0110chplr44g%2F5da96c03

KeyboardInterrupt: 